# December 21, 2021

https://adventofcode.com/2021/day/21

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import datetime
import re

In [2]:
def pnow():
    print( datetime.datetime.now().isoformat() )

In [3]:
with open("../data/2021/21.txt", "r") as f:
    text = f.read()

data = [ int(re.search("\d+$", line)[0]) for line in text.split("\n")]

# Part 1

In [4]:
def mod1( x, base ):
    '''x % [base] but where class 0 is called [base]'''
    return ( (x-1) % base + 1)

In [5]:
def roll( turn ):
    dice = 6 + (turn-1)*9
    return mod1(dice, 300)

def part1( start ):
    player1 = {"pos":start[0], "score":0}
    player2 = {"pos":start[1], "score":0}
    turn = 0

    while player1["score"] < 1000 and player2["score"] < 1000:
        turn += 1
        r = roll(turn)
        player1["pos"] = mod1( player1["pos"] + r, 10 )
        player1["score"] += player1["pos"]

        if player1["score"] < 1000:
            turn += 1
            r = roll(turn)
            player2["pos"] = mod1( player2["pos"] + r, 10 )
            player2["score"] += player2["pos"]

        #print("p1", player1)
        #print("p2", player2)

    # Done!
    if player1["score"] >= 1000:
        return player2["score"] * turn * 3
    else:
        return player1["score"] * turn * 3


In [6]:
part1([4,8])

739785

In [7]:
part1(data)

684495

# Part 2

Use Kolmogorov Backward Step Equation

Start with probability because it's easier for me to think about than counting universes

In [8]:
def prob_dist_to_goal( given, pos, goal=21, nsides=3, nspaces=10 ):
    '''return dict where [value] is probability of reaching goal in [key] turns'''
    if given >= goal:
        return{0:1.0}
    
    full_dict = {}
    for r in range(nsides):
        new_pos = mod1(pos+r+1, nspaces)
        next_dict = prob_dist_to_goal( given + new_pos, new_pos, goal, nsides, nspaces )
        for k in next_dict.keys():
            if k+1 in full_dict:
                full_dict[k+1] += next_dict[k]/nsides
            else:
                full_dict[k+1] = next_dict[k]/nsides

    return full_dict

Okay.... count the number of universes that take X turns

In [9]:
def count_dist_to_goal( given, pos, goal=21, nsides=3, nspaces=10 ):
    '''return dict where [value] is number of paths to goal in [key] turns'''

    if given >= goal:
        return{0:1.0}
    
    full_dict = {}
    for r in range(nsides):
        new_pos = mod1(pos+r+1, nspaces)
        next_dict = count_dist_to_goal( given + new_pos, new_pos, goal, nsides, nspaces )
        for k in next_dict.keys():
            if k+1 in full_dict:
                full_dict[k+1] += next_dict[k] + 1
            else:
                full_dict[k+1] = next_dict[k]+1
    
    return full_dict

Nertz! Can't do players independently because universe stops splitting as soon as one finishes.  
The other player will have fewer universes

In [10]:
def get_roll_table( n, d ):
    roll_table = { pips+1:1 for pips in range(d) }

    for roll in range(n-1):
        next_table = defaultdict(int)

        for pips in range(d):
            for tot in roll_table.keys():
                next_table[ tot+pips+1 ] += roll_table[ tot ]
        roll_table = next_table.copy()

    return roll_table

def bi_count_dist_to_goal( score1, score2, pos1, pos2, who, roll_table, goal=21, nspaces=10 ):
    '''
    Return the number of universes where players win, given they have
    
    score1/2: current scores for players
    pos1/2: current positions for players
    who: current player (1 or 2)
    goal: target score
    nsides: number of die sides, assumed 1..nsides
    nspaces: number of spaces on track, assumed 1..nspaces
    '''

#    print(f'''{who}: {score1} @{pos1} --- {score2} @{pos2}''')

    # Base Cases:
    # It's player 2's turn but player 1 already won!
    if who == 2 and score1 >= goal:
        return [1,0]
    # It's player 1's turn but player 2 already won!
    if who == 1 and score2 >= goal:
        return [0,1]
    # not a bas case, but we already solved it:
    # memoization is init outside this function in global space ><
    if (score1, score2, pos1, pos2, who) in bad_style_dict.keys():
        return bad_style_dict[(score1, score2, pos1, pos2, who) ]
    
    counts = [0,0]
    for roll, count in roll_table.items():
        #print("further on:", roll count)
        if who == 1:
            new_pos = mod1( pos1 + roll, nspaces )
            future = bi_count_dist_to_goal( score1+new_pos, score2, new_pos, pos2, 2, roll_table, goal, nspaces )
        else:
            new_pos = mod1( pos2 + roll, nspaces )
            future = bi_count_dist_to_goal( score1, score2+new_pos, pos1, new_pos, 1, roll_table, goal, nspaces )

        # count the possible future universes given roll of r
#        print("return from ", roll, count)
#        print("FUTURE = ", future)
#        print("COUNT = ", count)
        counts[0] += future[0]*count
        counts[1] += future[1]*count
#        print("COUNTS = ", counts)
#        print("===========================")
    bad_style_dict[ (score1, score2, pos1, pos2, who) ] = counts
#    print( len(bad_style_dict.keys()) )

    return counts


In [11]:
roll_table = get_roll_table(3,3)
roll_table

defaultdict(int, {3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1})

In [12]:
%%time
bad_style_dict = {}
counts = bi_count_dist_to_goal( 0, 0, 4, 8, 1, roll_table, goal=21, nspaces=10 )

CPU times: total: 93.8 ms
Wall time: 92.3 ms


In [13]:
counts

[444356092776315, 341960390180808]

In [14]:
%%time
bad_style_dict = {}
counts = bi_count_dist_to_goal( 0, 0, data[0], data[1], 1, roll_table, goal=21, nspaces=10 )

CPU times: total: 78.1 ms
Wall time: 84.2 ms


In [15]:
counts

[138289532619163, 152587196649184]

In [16]:
max(counts)

152587196649184